**Driver Code**

In [2]:
import os
import re
import tkinter as tk
import pandas as pd
from sklearn.cluster import KMeans
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Get the path of the script file
script_path = os.path.abspath("Doc50")
# Get the directory containing the script file
script_dir = os.path.dirname(script_path)
# Combine the script directory with the relative path to the Doc50 folder
doc_folder = os.path.join(script_dir, "Doc50")


stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

# Preprocessing
def preprocess_doc(doc):
    # Convert to lowercase
    doc = doc.lower()
    # Remove email addresses
    doc = re.sub(r'\S+@\S+', '', doc)
    # Remove *
    doc = re.sub(r'\*', '', doc)
    # Remove HTML tags
    doc = re.sub(r'<[^>]+>', '', doc)
    # Remove punctuation
    doc = re.sub(r'[^\w\s]', '', doc)
    doc = re.sub(r'\d+', '', doc)
    doc = re.sub(r'_', '', doc)
    # Tokenize
    words = doc.split()
    # Remove stop words
    words = [w for w in words if w not in stop_words]
    # Perform stemming or lemmatization
    # words = [stemmer.stem(w) for w in words]
    words = [lemmatizer.lemmatize(w) for w in words]  # uncomment for lemmatization
    # Join words back into a string
    doc = ' '.join(words)
    return doc

docs = []

def preprocess_and_train(doc_folder):
   
    for file in os.listdir(doc_folder):
        try:
            with open(os.path.join(doc_folder, file), 'r', encoding='utf-8', errors='ignore') as f:
                doc = f.read()
                doc = preprocess_doc(doc)
                docs.append(doc)
                # print(doc)
        except Exception as e:
            print(f"Error reading file {file}: {e}")

    if not docs:
        print("No documents were read into the DataFrame.")
    else:
        df = pd.DataFrame({'text': docs})

    # Train Word2Vec model
    sentences = [doc.split() for doc in docs]
    model = Word2Vec(sentences=sentences, vector_size=100, window=5, min_count=5, workers=4)

    return df, model

df, model = preprocess_and_train(doc_folder)


# Word2Vec-based features
X_w2v = []
for doc in docs:
    w2v_doc = []
    for word in doc.split():
        try:
            w2v = model.wv[word]
            w2v_doc.append(w2v)
        except KeyError:
            pass
    if w2v_doc:
        w2v_doc_mean = sum(w2v_doc) / len(w2v_doc)
        X_w2v.append(w2v_doc_mean)
    else:
        X_w2v.append([0] * 50)
X_w2v = pd.DataFrame(X_w2v)

# Baseline I: TF-based features with bigrams
count_vectorizer = CountVectorizer(min_df=2, max_df=0.9, ngram_range=(1,2))
X_count = count_vectorizer.fit_transform(df['text'])


# Baseline II: TF-IDF features
tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(df['text'])

def calculate_purity(y_true, y_pred, k):
    total_purity = 0
    for i in range(k):
        cluster_indices = [j for j in range(len(y_true)) if y_pred[j] == i]
        cluster_labels = [y_true[j] for j in cluster_indices]
        most_common_label = max(set(cluster_labels), key=cluster_labels.count)
        cluster_purity = cluster_labels.count(most_common_label) / len(cluster_labels)
        total_purity += cluster_purity * len(cluster_indices)
    return total_purity / len(y_true)

true_labels = []
documents = []
# Iterate through each subfolder of the ground truth folder
for folder_name in os.listdir("Doc50 GT"):
    folder_path = os.path.join("Doc50 GT", folder_name)
    if os.path.isdir(folder_path):
        # Iterate through each file in the subfolder
        for file_name in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file_name)
            if os.path.isfile(file_path):
                # Extract the label from the file name (the first character)
                true_label = int(file_name[0])
                true_labels.append(true_label)
                # Read the contents of the file and preprocess it
                with open(file_path, 'r', encoding='utf-8') as f:
                    contents = f.read()
                preprocessed_contents = preprocess_doc(contents)
                documents.append(preprocessed_contents)

class Window(tk.Tk):
    def __init__(self):
        super().__init__()
        self.initUI()

    def initUI(self):
        # create the text box for output
        self.output_text = tk.Text(self)
        self.output_text.config(state=tk.DISABLED)

        # create the layout for the window
        vbox = tk.Frame(self)
        self.output_text.pack(side=tk.TOP, fill=tk.BOTH, expand=True)
        vbox.pack(fill=tk.BOTH, expand=True)

        # call the function that prints the output
        self.print_output()

        # set up event handling for window closing
        self.protocol("WM_DELETE_WINDOW", self.on_exit)

    def on_exit(self):
        self.destroy()

    def print_output(self):
        # Baseline I
        k = 10
        km = KMeans(n_clusters=k, init='k-means++', max_iter=100, n_init=1)
        km.fit(X_count)
        y_pred = km.labels_
        purity = calculate_purity(true_labels, y_pred, k)
        self.output_text.config(state=tk.NORMAL)
        self.output_text.insert(tk.END, f"Purity score (Baseline I): {purity}\n")
        self.output_text.config(state=tk.DISABLED)

        # Baseline II
        km.fit(X_tfidf)
        y_pred = km.labels_
        purity = calculate_purity(true_labels, y_pred, k)
        self.output_text.config(state=tk.NORMAL)
        self.output_text.insert(tk.END, f"Purity score (Baseline II): {purity}\n")
        self.output_text.config(state=tk.DISABLED)

        # Word2Vec
        k = 10
        km = KMeans(n_clusters=k, init='k-means++', max_iter=100, n_init=1)
        km.fit(X_w2v)
        y_pred = km.labels_
        purity = calculate_purity(true_labels, y_pred, k)
        self.output_text.config(state=tk.NORMAL)
        self.output_text.insert(tk.END, f"Purity score (Word2Vec embeddings): {purity}\n")
        self.output_text.config(state=tk.DISABLED)

if __name__ == '__main__':
    window = Window()
    window.mainloop()

